In [3]:
import datetime, json
from engine.helpers import write_file
from engine.audio import load_audio
from engine.metadata import metadata_db
from engine.settings import BSC_MS_DATETIME_FORMAT

mdb = metadata_db('metadata.json')
load_audio(mdb, 'audio').split_audio_into_peaks('manual_peak_1', .2, .2).run()

newMdb = dict()
for stem in mdb.db:
	entry = mdb.db[stem]
	time = datetime.datetime.strptime(entry.date + ' ' + entry.time, '%Y-%m-%d %H:%M:%S').timestamp()

	for peak in entry.peaks:
		timestamp = peak['timestamp']
		sequence = peak['sequence']
		peakTime = time + timestamp

		newEntry = dict()
		newEntry['datetime'] = datetime.datetime.fromtimestamp(peakTime).strftime(BSC_MS_DATETIME_FORMAT)
		newEntry['timestamp'] = peakTime
		newEntry['peaks'] = [0.2]
		newEntry['label'] = sequence
		newEntry['individual'] = entry.individual
		newEntry['colony'] = entry.colony
		newEntry['id'] = entry.id

		newEntry['duration'] = 0.4

		newMdb[f'{stem}_peak{timestamp * 1000:.0f}_{sequence}'] = newEntry


write_file(f"../data/manual_peak_1/metadata.json", json.dumps(newMdb, indent=2))

Loaded audio tracks from audio
Already processed: manual_peak_1\audio\2015-06-08_ID8_0026_peak140_OUT.wav, manual_peak_1\audio\2015-06-08_ID8_0026_peak1714_VS.wav, manual_peak_1\audio\2015-06-08_ID8_0026_peak1784_VSV.wav, ... (11989 others)


create train & validation set for the min20 set

In [1]:
from utils.dataset import create_mp1_data_set
create_mp1_data_set('min20', lambda elem: elem[1] > 20)

Log for interactive will be written in log-2020-05-07_14-51-18


create train & validation set for the min100 set

In [1]:
from utils.dataset import create_mp1_data_set
create_mp1_data_set('min100', lambda elem: elem[1] > 100)


Log for interactive will be written in log-2020-04-30_15-45-41


search all labeld wav files and join them with Vermessene_Sil_Babbeln_2020.xlsx

In [4]:
import json, itertools, glob
import pandas as pd
from pathlib import Path
from os.path import basename
from collections import defaultdict


wav_index = defaultdict(list)
for file in Path(r'C:\OneDrive\unibe\master\masterarbeit\data\evalSyllables').glob("**/*.wav"):
	key = basename(file)
	wav_index[key].append(file)

	if len(wav_index[key]) > 1:
		print(wav_index[key])

full_df = pd.read_excel(r'../data/tmp/Vermessene_Sil_Babbeln_2020.xlsx', sheet_name=None)

df = pd.DataFrame()
for name, sheet in full_df.items():
    sheet['sheet'] = name
    sheet = sheet.rename(columns=lambda x: x.split('\n')[-1])
    df = df.append(sheet[['syllable_type', 'file_name']])

source_list = list()
for index, row in df.iterrows():
	filename = str(row['file_name']) + '.wav'
	if filename in wav_index:
		source_list.append(wav_index[filename][0])
	else:
		source_list.append('not found: ' + filename)

df.insert(2, "path", source_list, True)


write_file(f"../data/manual_peak_1/metadata.json", json.dumps(newMdb, indent=2))

,syllable_type,file_name,path
0,D1,BCI_SL1_Jt1unbe_15_06_15_017_D1,C:\OneDrive\un...
1,D1,BCI_SL1_Jt1unbe_15_06_15_027_D1,C:\OneDrive\un...
2,D1,BCI_SL1_Jt1unbe_15_06_08_029_D1,C:\OneDrive\un...
3,D1,BCI_SL1_Rblau-Lweisshellblau_15_06_18_008_D1,C:\OneDrive\un...
4,D1,BCI_SL1_Rblau-Lweisshellblau_15_06_18_008_D1_(2),C:\OneDrive\un...
...,...,...,...
247,C1,22,not found: 22.wav
248,C1,22,not found: 22.wav
249,C2,81,not found: 81.wav
250,C2,51,not found: 51.wav
